# Prep 1: steps to install the packages

In [2]:
!pip install py_entitymatching
!pip install scipy
!pip install numpy
!pip install pandas

    100% |████████████████████████████████| 2.0MB 831kB/s eta 0:00:01
    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/tmp/pip-build-l428MW/pandas-profiling/setup.py", line 13, in <module>
        with open(path.join(this_directory, 'README.md'), encoding='utf-8') as f:
    TypeError: 'encoding' is an invalid keyword argument for this function
    
    ----------------------------------------
Command "python setup.py egg_info" failed with error code 1 in /tmp/pip-build-l428MW/pandas-profiling/
    100% |████████████████████████████████| 24.8MB 69kB/s  eta 0:00:011
    100% |████████████████████████████████| 17.0MB 100kB/s eta 0:00:011
  Using cached https://files.pythonhosted.org/packages/e1/72/179a78b565ecf01fe98dab6417581d30acac15c2d93c49f93169ebea99b1/numpy-1.16.3-cp27-cp27mu-manylinux1_x86_64.whl
    100% |████████████████████████████████| 10.1MB 170kB/s eta 0:00:01
  Using cach

# Prep 2: enter the file location on your harddisk

In [4]:
table_a = '<table_a_location_downloaded_from_cloudmatcher_loc>'
table_b = '<table_a_location_downloaded_from_cloudmatcher_loc>'
candidate_set = '<candidate_set_downloaded_from_cloudmatcher_loc>'
prediction_set = '<prediction_set_downloaded_from_cloudmatcher_loc>'

# Prep 3: reading the files into pandas dataframe

In [ ]:
dfa = pd.read_csv(table_a)
dfb = pd.read_csv(table_b)
dfc = pd.read_csv(candidate_set)
dfp = pd.read_csv(prediction_set)

# Module: debug_blocker
# Description: debug the blocking rule using the below script to ensure you are not dropping true matches
# Note: You need to run Prep 1 and 2 in order to run this

In [35]:
# Example input format:
# Format of table_a:
# _id, attribute1, attribute2, ....., attributen

# Format of table_b:
# _id, attribute1, attribute2, ....., attributen

# Format of candidate_set
# A_id,B_id
# where A_id is _id from table_a and B_id is the _id column value from table_b

In [33]:
import py_entitymatching as em
import pandas as pd

def run_debug_blocker(table_a, table_b, table_a_key, table_b_key, candidate_set):
    dfl = em.read_csv_metadata(table_a, key=table_a_key)
    dfr = em.read_csv_metadata(table_b, key=table_b_key)

    # reading the candidate set and adding key
    dfcand = pd.read_csv(candidate_set)
    dfcand.drop_duplicates(inplace=True)
    dfcand.to_csv('cand_set_with_index.csv', index_label='id')

    dfcset = em.read_csv_metadata('cand_set_with_index.csv', key='id', ltable=dfl, 
                                  rtable=dfr, fk_ltable='A_id', fk_rtable='B_id')

    # running debug blocker to identify the records in A x B \ C
    debug_file = em.debug_blocker(dfcset, dfl, dfr)
    
    return debug_file

In [30]:
debug_file = run_debug_blocker(table_a, table_b, table_a_key, table_b_key, candidate_set)

# Module: estimate_precision_recall
# Description: the below code helps you get an estimation of P/R on the candidate set.

In [80]:
import pandas as pd
from scipy.stats import norm
from numpy import sqrt

delta = .05
Z = norm.ppf(1 - (delta / 2))

def estimate_PR(labeled_pairs, reduced_cands, predicted_matches):
    '''
    labeled_pairs - a pandas dataframe with schema id1,id2,label
                    Note label needs to be Boolean

    reduced_cands - a pandas dataframe with schema id1,id2
    predicted_matches - a pandas dataframe with schema id1,id2
    
    return:
        ( (recall lower bound, recall upper bound), (precision lower bound, precision upper bound) )
    '''

    reduced_cand_set = set(zip(reduced_cands.id1, reduced_cands.id2))
    predicted_matches = set(zip(predicted_matches.id1, predicted_matches.id2))
    
    # estimate the recall
    # number of positives in the labeled sample
    actual_pos = float(labeled_pairs.label.sum())
    # the maximum number of postives in the candidate set
    max_actual_pos = float(actual_pos + len(reduced_cand_set) - len(labeled_pairs))
    
    # true positives in the labeled sample
    true_pos = float(labeled_pairs.apply(lambda x : (x['id1'], x['id2']) in predicted_matches and x['label'], axis=1).sum())
    #estimated recall
    recall = float(true_pos / actual_pos)

    recall_error = Z * sqrt( ((recall * (1 - recall)) / actual_pos) * ((max_actual_pos - actual_pos) / max_actual_pos - 1) )


    # estimate Precision
    labeled_set  = set(zip(labeled_pairs.id1, labeled_pairs.id2))
    predicted_pos = float(len(labeled_set & predicted_matches))
    
    predicted_pos_in_reduced_cand_set = float(len(reduced_cand_set & predicted_matches))
    
    alpha =  predicted_pos_in_reduced_cand_set / len(predicted_matches)
    precision = alpha * (true_pos / predicted_pos)
    
    precision_error = alpha * Z * sqrt( ((precision * (1 - precision)) / predicted_pos) * (float((len(predicted_matches) - predicted_pos)) / (len(predicted_matches)  - 1)) )

    return ((recall - recall_error, recall + recall_error),
            (precision - precision_error, precision + precision_error))

In [83]:
print(estimate_PR(labeled, reduced_cand_set, dfp))